# Procurement Endpoint - Batch Calculations

* Please make a copy of this file, otherwise other users might be able to access your API key.
* Run the individual codes snippets one-by-one and make sure to follow the instructions.
* There are several placeholders in the codes (all in capital letters). Make sure to replace these with the required values.



# 1 - Initializations

In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import json
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
# If you are running this script in Google Colab, you can upload the file under the folder icon in the navigation bar on the left side
file_path ="INPUT_FILE_PATH"

procurement_data_raw = pd.read_csv(file_path)

In [ ]:
api_key="API_KEY"
authorization_headers = {"Authorization": f"Bearer {api_key}"}

url = "https://api.climatiq.io/procurement/v1/spend/batch"
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

To make sure the following code knows which columns to use as input, we need to rename the columns of your data. Please replace the placeholders with the columns names as defined in your input data.
Ensure that the currency has the following format: eur, usd, gbp, etc.
Ensure that the spend region uses ISO codes: DE, GB, US, ...

In [ ]:
input_table = procurement_data_raw
input_table=input_table.rename(columns={'SPEND_AMOUNT': 'money',
                            'MAPPED_ACTIVITY': 'activity_id',
                            'SPEND_YEAR': 'spend_year',
                            'CURRENCY': 'money_unit',
                            'SPEND_REGION': 'spend_region'})
input_table['money_unit'] = input_table['money_unit'].str.lower()
input_table['money'] = input_table['money'].apply(float)

You can either use industry classification codes (NACE, UNSPSC or MCC) for the identification of the right emission factor or the activity_id. Depening on your approach, run one of the following two codes:


In [ ]:
#Run this is you have mapped your spend against EXIOBASE actvities - Please replace the placeholders with the columns names as defined in your input data
input_table=input_table.rename(columns={ 'MAPPED_ACTIVITY': 'activity_id'})

In [ ]:
#Run this is you have mapped your spend against industry codes - make sure to replace the placeholders
input_table['classification_type'] = 'USED_SCHEME' #unspsc, nace2 or mcc
input_table=input_table.rename(columns={ 'INDUSTRY_CODE': 'classification_code'})

##2 - Currency Check
Climatiq supports many different currencies, but we still need to check that your data does not contain any currencies not support by the API and if so, ensure we apply a conversion factor to the data.

In [ ]:
list_of_available_currencies = ["usd","afn","dzd","ars","aud","bhd","brl","cad","kyd","cny","dkk","egp","eur","hkd","huf","isk","inr","iqd","ils","jpy","lbp","mxn","mad","nzd","nok","qar","rub","sar","sgd","zar","krw","sek","chf","thb","twd","tnd","try","aed","gbp"]
used_currencies = procurement_data_raw["Currency"]
used_currencies = used_currencies.drop_duplicates().str.lower().reset_index()["Currency"].values.tolist()
currencies_to_be_converted = list(set(used_currencies) - set(list_of_available_currencies))

#List of currencies to be converted
currencies_to_be_converted

['pkr', 'mop', 'myr', 'xof', 'idr', 'vnd', 'ngn', 'pln', 'kes', 'cop', 'mur']

If the array above is empty, you can skip the remaining steps for the currency check.
If these are currencies show, please make sure to enter the conversion rates to EUR into the array below.

In [ ]:
conversions = pd.DataFrame({"factor" :[CONVERSION_RATE_1,CONVERSION_RATE_2,CONVERSION_RATE_3,...]}, index = currencies_to_be_converted)
conversions

,factor
pkr,0.003200
mop,0.120000
myr,0.190000
xof,0.000150
idr,0.000060
vnd,0.000037
ngn,0.000740
pln,0.230000
kes,0.007100
cop,0.000230


In [ ]:
#Applying the conversions
for index, row in conversions.iterrows():
    input_table.loc[input_table.money_unit==index,'money']*=row['factor']
    input_table.loc[input_table['money_unit'] == index, 'money_unit'] = 'eur'

#3 - Calling the API in batches

In [ ]:
results = pd.DataFrame()
for i in range(0, input_table.size, 100):

    subset = input_table.iloc[i:i + 100]

    if 'classification_type' in input_table.columns:
      json_data = subset.apply(lambda row: {
          'money': row['money'],
          'spend_region': row['spend_region'],
          'spend_year': row['spend_year'],
          'money_unit': row['money_unit'],
          'activity': {
              'classification_type': row['classification_type'],
              'classification_code': row['classification_code']
          }
      }, axis=1).to_json(orient='records')
    else:
      json_data = subset.apply(lambda row: {
          'money': row['money'],
          'spend_region': row['spend_region'],
          'spend_year': row['spend_year'],
          'money_unit': row['money_unit'],
          'activity': {
              'activity_id': row['activity_id']
          }
      }, axis=1).to_json(orient='records')
    jsonFormat = json.loads(json_data)
    response = session.post(url, json=jsonFormat, headers=authorization_headers)
    data = response.json()
    data = pd.json_normalize(data['results'])
    results = pd.concat([results, data], axis=0)

In [ ]:
results

,notices,estimate.co2e,estimate.co2e_unit,estimate.co2e_calculation_method,estimate.co2e_calculation_origin,estimate.emission_factor.name,estimate.emission_factor.activity_id,estimate.emission_factor.id,estimate.emission_factor.access_type,estimate.emission_factor.source,...,estimate.activity_data.activity_value,estimate.activity_data.activity_unit,estimate.audit_trail,calculation_details.tax_margin,calculation_details.trade_margin,calculation_details.transport_margin,calculation_details.inflation_applied,error,error_code,message
0,[],28.040,kg,ar5,source,Education services,education-type_education_services,38fc2d26-9350-4237-972c-7fe513359a07,public,EXIOBASE,...,567.70,eur,enabled,0.0,0.000000,0.000000,0.026414,NaN,NaN,NaN
1,[],288.100,kg,ar5,source,Other business services,professional_services-type_other_business_serv...,323b1c60-faf1-4e70-9f9e-c0529fa3dde9,public,EXIOBASE,...,1709.00,eur,enabled,0.0,0.000000,0.000000,0.150363,NaN,NaN,NaN
2,[],1.132,kg,ar5,source,Hotel and restaurant (services),restaurants_accommodation-type_hotel_restauran...,4334df10-a32a-4175-af4a-1643f746184f,public,EXIOBASE,...,11.15,eur,enabled,0.0,0.000000,0.000000,0.076248,NaN,NaN,NaN
3,[],32848.000,kg,ar5,source,Research and development services,professional_services-type_research_developmen...,c4ecd227-e018-4f36-8f2e-476c1ca24640,public,EXIOBASE,...,150266.00,eur,enabled,0.0,0.000000,0.000000,0.116924,NaN,NaN,NaN
4,[],702.200,kg,ar5,source,Post and telecommunication,communication_services-type_post_telecommunica...,9ec3bdd2-6231-4e60-9c6a-89ac819b4852,public,EXIOBASE,...,7641.00,eur,enabled,0.0,0.000000,0.000000,-0.017684,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,[],14.270,kg,ar5,source,Food products (not elsewhere specified),consumer_goods-type_food_products_not_elsewher...,c138c22c-7d37-4fcc-b052-3438f6b494fe,public,EXIOBASE,...,24.61,eur,enabled,0.0,0.137648,0.014553,0.205558,NaN,NaN,NaN
4,[],105.200,kg,ar5,source,Food products (not elsewhere specified),consumer_goods-type_food_products_not_elsewher...,c138c22c-7d37-4fcc-b052-3438f6b494fe,public,EXIOBASE,...,181.40,eur,enabled,0.0,0.137648,0.014553,0.205558,NaN,NaN,NaN
5,[],150.100,kg,ar5,source,Insurance and pension funding services (except...,insurance-type_insurance_pension_funding_servi...,149a98b1-f20c-4ef9-8569-25e214fb1f9b,public,EXIOBASE,...,1974.00,eur,enabled,0.0,0.000000,0.000000,0.032200,NaN,NaN,NaN
6,[],4453.000,kg,ar5,source,Computer and related services,professional_services-type_computer_related_se...,a7eed7c6-3c48-4854-98b3-4bc199832356,public,EXIOBASE,...,96808.00,eur,enabled,0.0,0.000000,0.000000,0.026414,NaN,NaN,NaN


#4 - Exporting the results
Adjust the file path and name if needed. The file will show up on the left-hand side in the file explorer and can simply be downloaded to your machine.

In [ ]:
results = results.reset_index(drop=True)
procurement_results = pd.concat([procurement_data_raw, results], axis=1)
procurement_results.to_csv('ProcurementEmissions.csv', index=False)